In [4]:
!pip install ultralytics

  Using cached ultralytics_thop-2.0.11-py3-none-any.whl (26 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached tbb-2021.13.1-py3-none-win_amd64.whl (286 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
from ultralytics import YOLO


In [5]:
model = YOLO("C:/Users/kirub/OneDrive/Desktop/buildingmodel/BUILDINGSEG.pt")

In [13]:
video_path = "C:/Users/kirub/OneDrive/Desktop/buildingmodel/input/building4.mp4"

results = model.predict(source=video_path, save=True, project="C:/Users/kirub/OneDrive/Desktop/buildingmodel/videooutput", name='output')



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1860) C:\Users\kirub\OneDrive\Desktop\buildingmodel\input\building4.mp4: 384x640 (no detections), 74.0ms
video 1/1 (frame 2/1860) C:\Users\kirub\OneDrive\Desktop\buildingmodel\input\building4.mp4: 384x640 (no detections), 75.9ms
video 1/1 (frame 3/1860) C:\Users\kirub\OneDrive\Desktop\buildingmodel\input\building4.mp4: 384x640 (no detections), 77.6ms
video 1/1 (frame 4/1860) C:\Users\kirub\OneDrive\Desktop\buildingmodel\input\building4

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [23]:
from ultralytics import YOLO
import cv2
import os



# Video input and output paths
input_video_path = "C:/Users/kirub/OneDrive/Desktop/buildingmodel/input/building.mp4"  # Replace with your input video path
output_folder = r"C:/Users/kirub/OneDrive/Desktop/buildingmodel/videooutput"
output_video_path = os.path.join(output_folder, "output_video.mp4")

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Pixel-to-centimeter ratio (set this based on your context or calibration)
pixel_to_cm_ratio = 0.026  # Example: 1 pixel = 0.026 cm

# Load video
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform instance segmentation
    results = model.predict(source=frame, imgsz=640, conf=0.5, task="segment", save=False, stream=True)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x1, y1, x2, y2)
        masks = result.masks.data.cpu().numpy()  # Instance masks
        classes = result.boxes.cls.cpu().numpy()  # Class IDs

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box
            width_px = x2 - x1
            height_px = y2 - y1

            # Convert measurements to centimeters
            width_cm = width_px * pixel_to_cm_ratio
            height_cm = height_px * pixel_to_cm_ratio

            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Display width and height in cm
            cv2.putText(frame, f"W: {width_cm:.2f} cm H: {height_cm:.2f} cm", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

            # Resize and process the mask
            mask = masks[i]
            mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
            mask_binary = (mask_resized > 0.5).astype("uint8")  # Binary mask

            # Create a mask overlay
            mask_overlay = cv2.merge([mask_binary * 255] * 3)  # Convert to 3 channels
            colored_mask = cv2.addWeighted(frame, 1, mask_overlay, 0.3, 0)  # Blend mask with transparency
            frame = cv2.bitwise_and(frame, colored_mask)  # Combine with the frame

    # Write the processed frame to the output video
    out.write(frame)

    # Display the frame (optional, press 'q' to quit)
    cv2.imshow("YOLOv8 Instance Segmentation", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at: {output_video_path}")



0: 384x640 2 spallings, 69.0ms
Speed: 4.7ms preprocess, 69.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spallings, 76.5ms
Speed: 5.4ms preprocess, 76.5ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 spalling, 46.4ms
Speed: 4.8ms preprocess, 46.4ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 spalling, 48.6ms
Speed: 5.4ms preprocess, 48.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spallings, 46.7ms
Speed: 3.8ms preprocess, 46.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spallings, 55.1ms
Speed: 5.5ms preprocess, 55.1ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spallings, 23.9ms
Speed: 2.5ms preprocess, 23.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 spallings, 24.3ms
Speed: 6.0ms preprocess, 24.3ms inference, 2.6ms postprocess per i

In [68]:
import cv2
import numpy as np
from ultralytics import YOLO


# Load the video
cap = cv2.VideoCapture("C:/Users/kirub/OneDrive/Desktop/buildingmodel/input/real1.mp4")  # Use the path to your video file

# Get the video frame width, height, and FPS (Frames per second) for the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the output video path and codec
output_path = r"C:/Users/kirub/OneDrive/Desktop/buildingmodel/videooutput/real01.mp4"  # Set the desired output video path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Define a fixed scale factor (cm per pixel) based on your real-world measurements
scale_cm_per_pixel = 0.03  # Example: 1 pixel = 0.1 cm (You need to adjust this based on your scene)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform inference
    results = model(frame)

    # Parse the results
    for result in results:
        # Extract instance segmentation data
        for det in result.boxes:
            # Get bounding box coordinates (x1, y1, x2, y2)
            x1, y1, x2, y2 = map(int, det.xyxy[0])  
            
            # Calculate the width and height in pixels
            width_px = x2 - x1
            height_px = y2 - y1

            # Convert the width and height to centimeters (scale is fixed)
            width_cm = width_px * scale_cm_per_pixel
            height_cm = height_px * scale_cm_per_pixel

            # Get the class name
            class_name = result.names[int(det.cls[0])]

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Display the class name and measurements (height, width in cm)
            label = f"{class_name} | H: {height_cm:.2f} cm | W: {width_cm:.2f} cm"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the frame to the output video
    out.write(frame)

    # Display the processed frame
    cv2.imshow('Video Feed', frame)

    # Press 'q' to quit the video window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer, and close the windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at {output_path}")



0: 288x640 1 spalling, 48.0ms
Speed: 683.0ms preprocess, 48.0ms inference, 15.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 58.1ms
Speed: 5.0ms preprocess, 58.1ms inference, 8.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 64.2ms
Speed: 3.4ms preprocess, 64.2ms inference, 8.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 40.4ms
Speed: 4.5ms preprocess, 40.4ms inference, 8.9ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 41.2ms
Speed: 0.0ms preprocess, 41.2ms inference, 9.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 39.8ms
Speed: 4.7ms preprocess, 39.8ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 46.2ms
Speed: 0.0ms preprocess, 46.2ms inference, 5.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 39.2ms
Speed: 1.6ms preprocess, 39.2ms inference, 9.1ms postprocess per image

In [ ]:
############################################### MEASUREMENT WITHOUT SEGMENTATION ###############################################################
import cv2
import numpy as np
from ultralytics import YOLO


# Load the video
cap = cv2.VideoCapture("C:/Users/kirub/OneDrive/Desktop/buildingmodel/input/real2.mp4") # Use the path to your video file

# Get the video frame width, height, and FPS (Frames per second) for the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the output video path and codec
output_path = r"C:/Users/kirub/OneDrive/Desktop/buildingmodel/videooutput/real2.mp4" # Set the desired output video path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 format
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Define the conversion factor for measurements (adjust this as per your real-world scaling)
# Assuming 1 pixel = 0.1 cm (this is just an example and needs to be calibrated)
pixel_to_cm = 0.07  # Adjust this value based on your real-world measurements

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference
    results = model(frame)

    # Parse the results
    for result in results:
        # Extract instance segmentation data (boxes and masks)
        for det in result.boxes:
            # Get bounding box coordinates (x1, y1, x2, y2)
            x1, y1, x2, y2 = map(int, det.xyxy[0])

            # Calculate width and height in pixels
            width = x2 - x1
            height = y2 - y1

            # Convert width and height to centimeters
            width_cm = width * pixel_to_cm
            height_cm = height * pixel_to_cm

            # Draw the bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Display the measurements (width and height in cm) on the frame
            cv2.putText(frame, f"W: {width_cm:.2f} cm", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.putText(frame, f"H: {height_cm:.2f} cm", (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Check if the result includes masks (segmentation)
            if hasattr(det, 'masks') and det.masks is not None:
                # Extract and display the mask
                mask = det.masks[0]  # Assuming only one mask per object
                mask = mask.cpu().numpy()  # Convert to numpy array
                mask = (mask * 255).astype(np.uint8)  # Convert to 255 for display
                colored_mask = cv2.applyColorMap(mask, cv2.COLORMAP_JET)  # Apply a color map to the mask

                # Apply the mask on the frame (overlay the mask on the bounding box area)
                frame = cv2.addWeighted(frame, 1, colored_mask, 0.5, 0)

            # Optionally, add text labels (class name, etc.)
            class_name = result.names[int(det.cls[0])]
            cv2.putText(frame, class_name, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the frame to the output video
    out.write(frame)

    # Display the processed frame
    cv2.imshow('Video Feed', frame)

    # Press 'q' to quit the video window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer, and close the windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at {output_path}")



0: 288x640 1 spalling, 59.7ms
Speed: 531.2ms preprocess, 59.7ms inference, 3.4ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 30.9ms
Speed: 0.0ms preprocess, 30.9ms inference, 7.5ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 16.0ms
Speed: 1.3ms preprocess, 16.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 9.1ms
Speed: 1.5ms preprocess, 9.1ms inference, 7.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 18.0ms
Speed: 2.0ms preprocess, 18.0ms inference, 3.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 4.8ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 17.7ms
Speed: 0.0ms preprocess, 17.7ms inference, 3.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 spalling, 19.5ms
Speed: 0.0ms preprocess, 19.5ms inference, 2.9ms postprocess per image at

In [3]:
#"C:/Users/kirub/OneDrive/Desktop/buildingmodel/input/real2.mp4"
#"C:/Users/kirub/OneDrive/Desktop/buildingmodel/videooutput/seg.mp4"
!pip install ultralytics opencv-python numpy

import cv2
import numpy as np
from ultralytics import YOLO



# Define a conversion factor (pixels to centimeters)
PIXEL_TO_CM_RATIO = 0.1  # Adjust based on your dataset and camera calibration

# Load the video
video_path = "C:/Users/kirub/OneDrive/Desktop/buildingmodel/input/real2.mp4" # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = "C:/Users/kirub/OneDrive/Desktop/buildingmodel/videooutput/seg.mp4"

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 instance segmentation
    results = model(frame, task='segment')

    annotated_frame = frame.copy()

    for result in results:
        masks = result.masks.data.cpu().numpy()  # Move tensor to CPU and convert to NumPy
        boxes = result.boxes.xyxy.cpu().numpy()  # Move tensor to CPU and convert to NumPy
        scores = result.boxes.conf.cpu().numpy()  # Move tensor to CPU and convert to NumPy
        classes = result.boxes.cls.cpu().numpy()  # Move tensor to CPU and convert to NumPy

        for mask, box, score, cls in zip(masks, boxes, scores, classes):
            # Convert the mask to binary and find contours
            binary_mask = (mask > 0.5).astype(np.uint8)  # Threshold mask
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw the segmentation mask and contours
            color = (0, 255, 0)  # Green color for masks
            for contour in contours:
                cv2.drawContours(annotated_frame, [contour], -1, color, 2)

                # Calculate the area in pixels
                area_pixels = cv2.contourArea(contour)

                # Convert the area to centimeters
                area_cm = area_pixels * PIXEL_TO_CM_RATIO**2

                # Draw the measurement on the frame
                M = cv2.moments(contour)
                if M["m00"] > 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    cv2.putText(annotated_frame, f"{area_cm:.2f} cm^2", (cX, cY),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Show the frame (optional)
    cv2.imshow("Annotated Video", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Annotated video saved to {output_path}")



NameError: name 'model' is not defined